Using the CNN weights that were provided by the paper authors. These create a .npy file called `forward_feats.npy`, which should be placed in `setup_existing_model`.

Run the script in `setup_existing_model` first

In [1]:
import glob as glob
import numpy as np
import pandas as pd
import os

In [2]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

# import numpy as np
# import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns


def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1

In [3]:
lis_feats = glob.glob('setup_existing_model/*feats.npy')
lis_feats

['setup_existing_model/nigeria_2013_forward_feats.npy',
 'setup_existing_model/mw_2016_forward_feats.npy',
 'setup_existing_model/tanzania_2011_forward_feats.npy',
 'setup_existing_model/uganda_2011_forward_feats.npy']

In [42]:
fname = lis_feats[2]
c = fname[fname.find('/')+1:fname.find('forward_feats.npy')-1]
feats = np.load(fname)
ims = os.listdir("process_data/data/ims_"+c)
# since we passed our images to the model in this order, we know index 0
# of this dataframe corresponds to index 0 of feats
df_im_raw = pd.DataFrame.from_dict({'images': ims}); df_im_raw.head()
print(df_im_raw.shape)
# this index corresponds to the row in "forward_feats.npy"
df_im_raw['feat_index'] = np.arange(len(df_im_raw))

(70677, 1)


In [43]:
im_to_cons = pd.read_csv('process_data/data/output/{}_full_guide.csv'.format(c))

In [44]:
# notice we have duplicate images that we only pass once to the model to reduce runtime load
# we need to add a column that tells us what index of feats to look at for that image
im_to_cons.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,check,clust_num,images_renamed
0,-4.866666,29.566666,-4.911112,29.611354,9.266667,3.680708,2,-4.866666_29.566665999999998.png,True,989,-4.866666_29.566665999999998_989.png
1,-4.875000,29.566666,-4.911112,29.611354,9.266667,3.680708,2,-4.875_29.566665999999998.png,True,989,-4.875_29.566665999999998_989.png
2,-4.883333,29.566666,-4.911112,29.611354,9.266667,3.680708,2,-4.883333_29.566665999999998.png,True,989,-4.883333_29.566665999999998_989.png
3,-4.891666,29.566666,-4.911112,29.611354,9.266667,3.680708,2,-4.891666000000001_29.566665999999998.png,True,989,-4.891666000000001_29.566665999999998_989.png
4,-4.900000,29.566666,-4.911112,29.611354,9.266667,3.680708,2,-4.9_29.566665999999998.png,True,989,-4.9_29.566665999999998_989.png


In [45]:
im_to_cons.shape

(112885, 11)

In [46]:
# now we can merge
im_to_cons = pd.merge(left=im_to_cons, right=df_im_raw, on='images')

In [47]:
im_to_cons.shape # we shouldn't lose any rows

(112885, 12)

In [49]:
group = im_to_cons.groupby(['clust_lat', 'clust_lon'])

In [50]:
num_clusts = len(group); num_clusts

1520

In [51]:
x = np.zeros((num_clusts, 4096))
y = []

In [52]:
# this goes through each cluster group and finds all images that are in the cluster
# it aggregates the features for those images across the cluster
for i, g in enumerate(group):
    lat, long = g[0]
    im_sub = im_to_cons[(im_to_cons['clust_lat'] == lat) & (im_to_cons['clust_lon'] == long)].reset_index(drop=True)
    agg_feats = np.zeros((len(im_sub), 4096))
    for j, d in im_sub.iterrows():
        agg_feats[j,:] = feats[d.feat_index]
    agg_feats = agg_feats.mean(axis=0) # averages the features across all images in the cluster
    
    x[i,:] = agg_feats
    y.append(g[1]['consumption'].values[0])

In [53]:
y = np.array(y)
y_log = np.log(y) # try predicting consumption and log consumption

In [54]:
x.shape

(1520, 4096)

In [55]:
_, _, _, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 1668.100537200059
best alpha 100000.0


0.0064997318728015575

In [56]:
_, _, _, r2 = predict_consumption(x, y)
r2

best alpha 1668.100537200059
best alpha 27.825594022071243
best alpha 100000.0
best alpha 27.825594022071243
best alpha 77.4263682681127


0.007776706039778575

<strong> Important Note </strong> <br>
If you read the original paper, you'll see that the paper's consumption prediction has r^2 of 0.33 or higher. Why is ours so much lower using their weights? Well, here we are using (free) 2019 images to predict their 2016 consumption values. Ideally, we should download 2016 images, and if we want to follow the paper exactly, we download 2013 images (for Malawi at least). However, this doesn't explain why if I train the model from scratch, it matches the paper's performance.